In [ ]:
from src.sampling.images import plot_frame_count_distributions

samples_root_dir = "data/samples"

plot_frame_count_distributions(samples_root_dir)

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11m-cls")
sota.initialize_model()
sota.train_model(optimizer="AdamW", lr0=0.0005)


#metrics = model.val(data="data/img/techniques/val")

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11n-cls")
sota.initialize_model()
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11m-cls")
sota.train_model()

In [3]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11n-cls")
#sota.train_model()
sota.test_model()

loading the model 'yolo11n-balance-50-155' with the weights at 'data/runs/sota/yolo11n-balance-50-155/train5/weights/best.pt'
Make sure to swap val and test splits for techniques_balanced, otherwise validation data will be used.
Ultralytics 8.3.78 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN RTX, 24576MiB)
YOLO11n-cls summary (fused): 47 layers, 1,534,991 parameters, 0 gradients, 3.2 GFLOPs
train: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/train... found 1893 images in 7 classes ✅ 
val: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/val... found 416 images in 7 classes ✅ 
test: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/test... found 894 images in 7 classes ✅ 


val: Scanning /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/val... 416 images, 0 corrupt: 100%|██████████| 416/416 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 26/26 [00:13<00:00,  1.95it/s]


                   all      0.892      0.998
Speed: 1.0ms preprocess, 21.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /mnt/c/Projects/HPE-POC/runs/classify/val12


In [5]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1")
hpednn.initialize_model()
hpednn.train_model()

loading the model 'arch1' from 'data/runs/hpe_dnn/arch1/train1/models/epoch_02__val_accuracy_0.7152.keras'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.631847  0.439228 -0.112943         0.967189         0.584897   
1  0.544295  0.383693  0.030225         0.970460         0.534586   
2  0.522651  0.319287  0.059222         0.973314         0.477302   
3  0.509914  0.345323  0.076138         0.975976         0.472843   
4  0.520337  0.300978  0.016012         0.978329         0.485842   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.497950        -0.257367                  0.999389      0.505804   
1         0.445929        -0.061585                  0.999447      0.459553   
2         0.356460        -0.093307                  0.999499      0.423448   
3         0.384584        -0.048136                  0.999547      0.429408   
4         0.349062        -0.114536                  0.999589      0.421399   

   

In [ ]:
%load_ext tensorboard

from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model()

In [4]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch2_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH2)
hpednn.train_model()

loading the model 'arch2_balanced' from 'data/runs/hpe_dnn/arch2_balanced/train2/models/epoch_12__val_accuracy_0.4639.keras'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.547458  0.463912 -0.324645         0.958448         0.555777   
1  0.529612  0.401783 -0.133231         0.962421         0.546691   
2  0.521964  0.327832 -0.046612         0.963201         0.486264   
3  0.507897  0.377151 -0.066129         0.964212         0.478969   
4  0.517631  0.282593  0.044080         0.967762         0.474001   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.487867        -0.281746                  0.998441      0.490354   
1         0.480926        -0.178477                  0.998549      0.466458   
2         0.366174        -0.130556                  0.998679      0.427865   
3         0.402826        -0.136893                  0.998766      0.430541   
4         0.337471        -0.096800                  0.998888    

In [6]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch3_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH3)
hpednn.train_model()

AttributeError: type object 'DnnArch' has no attribute 'ARCH3'

In [ ]:
type(func)

In [ ]:
%tensorboard --logdir data/runs/hpe_dnn/arch1_balanced/train1/logs/train